In [32]:
sku_number = int(input("Enter SKU Number: "))
day = int(input("Enter Day Number: "))

Enter SKU Number: 45
Enter Day Number: 343


In [33]:
import tensorflow as tf
from tensorflow.keras import layers
from collections import deque
import random
import numpy as np
import pandas as pd
from tqdm import tqdm  # For progress bar display

# Hyperparameters for Deep Q-Learning
discount_factor = 0.99
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
learning_rate = 0.001
batch_size = 64
memory_size = 2000
num_episodes = 3

# Dynamic state generation function
def get_state(row, feature_columns):
    # Collect all the feature values dynamically based on available columns
    state_values = [row[feature] for feature in feature_columns]
    return np.array(state_values).reshape(1, -1)

# Build the DQN model dynamically based on state size
def build_model(state_size, action_size):
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(state_size,)))
    model.add(layers.Dense(24, activation='relu'))
    model.add(layers.Dense(24, activation='relu'))
    model.add(layers.Dense(action_size, activation='linear'))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')
    return model

class DQNAgent:
    def __init__(self, state_size, action_size_bau, action_size_event):
        self.state_size = state_size
        self.action_size_bau = action_size_bau
        self.action_size_event = action_size_event
        self.memory = deque(maxlen=memory_size)
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay

        # Building the models with dynamic state size
        self.model_bau = build_model(state_size, action_size_bau)
        self.model_event = build_model(state_size, action_size_event)

    def remember(self, state, action, reward, next_state, done, event_type):
        self.memory.append((state, action, reward, next_state, done, event_type))

    def act(self, state, event_type):
        if np.random.rand() <= self.epsilon:
            if event_type == 'BAU':
                return random.randrange(self.action_size_bau)
            else:
                return random.randrange(self.action_size_event)
        else:
            if event_type == 'BAU':
                q_values = self.model_bau.predict(state)
                return np.argmax(q_values[0])
            else:
                q_values = self.model_event.predict(state)
                return np.argmax(q_values[0])

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)

        print("Training...")  # Simulated popup display
        with tqdm(total=len(minibatch)) as pbar:
            for state, action, reward, next_state, done, event_type in minibatch:
                target = reward
                if not done:
                    if event_type == 'BAU':
                        target = reward + discount_factor * np.amax(self.model_bau.predict(next_state)[0])
                        target_f = self.model_bau.predict(state)
                        target_f[0][action] = target
                        self.model_bau.fit(state, target_f, epochs=1, verbose=0)  # Hide verbose logs
                    else:
                        target = reward + discount_factor * np.amax(self.model_event.predict(next_state)[0])
                        target_f = self.model_event.predict(state)
                        target_f[0][action] = target
                        self.model_event.fit(state, target_f, epochs=1, verbose=0)  # Hide verbose logs
                pbar.update(1)  # Update the progress bar

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Pricing strategies
pricing_strategies = ["No Discount", "Small Discount", "Large Discount", "Price Skimming",
                      "Penetration Pricing", "Psychological Pricing", "Freemium Pricing"]

# Get discount values from the user for small and large discounts
def get_discount_values():
    small_discount = float(input("Enter the percentage for small discount (e.g., 5 for 5%): "))
    large_discount = float(input("Enter the percentage for large discount (e.g., 10 for 10%): "))
    return small_discount, large_discount

# Action mapping: Define what each action number means dynamically based on user input
def define_action_mapping(small_discount, large_discount):
    return {
        0: f"No discount (maintain price)",
        1: f"Small discount ({small_discount}% off)",
        2: f"Large discount ({large_discount}% off)",
        3: "Price increase (price skimming)",
        4: "Penetration Pricing (low price to enter market)",
        5: "Psychological Pricing (price just below a round number)",
        6: "Freemium Pricing (offer free tier with paid options)"
    }

# Initialize an empty dictionary to store results for SKUs and days
sku_day_pricing_map = {}

# Get discount values from user
small_discount, large_discount = get_discount_values()

# Define the action mapping based on user input for discount values
action_mapping = define_action_mapping(small_discount, large_discount)

# Final Pricing Decisions after training
def get_final_pricing(df, feature_columns):
    pricing_decisions = []
    for i in range(len(df)):
        state = get_state(df.iloc[i], feature_columns)
        event_type = df['Event_Type'][i]

        # Model predicts the best action (pricing strategy) dynamically
        action = agent.act(state, event_type)
        pricing_decisions.append(action)

    # Map actions to the action description
    df['Final_Pricing_Action'] = pricing_decisions
    df['Action_Description'] = df['Final_Pricing_Action'].map(action_mapping)
    return df

# Run through each episode for training
def run_episode(df, feature_columns):
    total_reward = 0
    for i in range(len(df)):
        state = get_state(df.iloc[i], feature_columns)
        event_type = df['Event_Type'][i]

        action = agent.act(state, event_type)

        next_state = get_state(df.iloc[i], feature_columns)
        reward = df['Revenue'][i]
        done = i == len(df) - 1

        agent.remember(state, action, reward, next_state, done, event_type)
        total_reward += reward

        if done:
            print(f"Episode finished with total reward: {total_reward}")
            break
    return total_reward

# Load the dataset
df = pd.read_csv(r"/content/final_sales_data_without_discount.csv")

# Define feature columns dynamically based on the dataset
feature_columns = ['Sales', 'Price']  # Adjust dynamically based on available features

# Initialize the agent with dynamic state size
agent = DQNAgent(state_size=len(feature_columns), action_size_bau=7, action_size_event=7)

# Training process
for e in range(num_episodes):
    total_reward = run_episode(df, feature_columns)
    print(f"Episode {e+1}/{num_episodes}, Total Reward: {total_reward}")
    agent.replay(batch_size)

# Get final pricing after training
trained_df = get_final_pricing(df, feature_columns)

# Display the results
#print(trained_df[['Product_ID', 'Day', 'Sales', 'Price', 'Event_Type', 'Action_Description']])

# Filter the DataFrame based on SKU number and day
temp = df[(df['Product_ID'] == sku_number)]
output = temp[(temp['Day'] == day)].iloc[-1]

# Print only the 'Action_Description' column
print(output['Action_Description'])


Enter the percentage for small discount (e.g., 5 for 5%): 5
Enter the percentage for large discount (e.g., 10 for 10%): 15
Episode finished with total reward: 97312397.2683147
Episode 1/3, Total Reward: 97312397.2683147
Training...


  0%|          | 0/64 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


  2%|▏         | 1/64 [00:01<01:32,  1.47s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


...

 98%|█████████▊| 63/64 [00:21<00:00,  2.44it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


100%|██████████| 64/64 [00:21<00:00,  2.94it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━